# MCMC - Gibbs Sampler

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import random
%matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [ ]:
fontsize = 40
scattersize=100
surf_labelpad = 50.

plt.style.use("seaborn")
plt.rcParams.update({'figure.figsize': (20,15), 'font.size': fontsize, 'axes.labelsize': fontsize, 'axes.labelpad': 15., 'text.usetex':True, 'xtick.labelsize': fontsize, 'xtick.major.pad': 20., 'ytick.labelsize': fontsize, 'ytick.major.pad': 20., })

In [ ]:
rng = np.random.RandomState(13)

## Example: Sample from multivariate Gaussian

In [ ]:
def gibbs_sampling(mus, sigmas, iters=10000):
    samples = np.zeros((iters, 2))
    y = np.random.rand() * 10

    for i in range(iters):
        # Insert your code here. Hint: You need to assing:
        # x = ?
        # y = ?

        samples[i, :] = [x, y]

    return samples

In [ ]:
target_mu = np.array([5, 5])
target_sigma = np.array([[1, .9], [.9, 1]])
iters=100

samples = gibbs_sampling(target_mu, target_sigma, iters=iters)

In [ ]:
sns.jointplot(x=samples[:, 0], y=samples[:, 1])
fig = plt.gcf()
fig.set_tight_layout(False)

Show how the Gibbs sampler moves:

In [ ]:
def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

In [ ]:
def animate(frame, fargs):
#     from cycler import cycler
    ax = fargs['ax']
    samples = fargs['samples']
    pos = fargs['pos']
    target_mu = fargs['target_mu']
    target_sigma = fargs['target_sigma']
    X = fargs['X']
    Y = fargs['Y']
    
    ax.cla()
    
    # Plot contour of target:
    Z = multivariate_gaussian(pos, target_mu, target_sigma)
    cset = ax.contour(X, Y, Z, cmap=cm.cividis, linewidths=0.5)
    ax.clabel(cset, inline=1, fontsize=10)

    # Plot history of samples:
    colors = [plt.cm.viridis(i) for i in np.linspace(0, 1, iters)]
    ax.set_prop_cycle(cycler('color', colors))
    h = []
    for t in range(frame):
        h = plt.plot(samples[t:t+2,0], samples[t:t+2,1], marker='o')
    ax.set_xlim(min(samples[:,0]), max(samples[:,0]))
    ax.set_ylim(min(samples[:,1]), max(samples[:,1]))
    plt.xlabel('$z_1$')
    plt.ylabel('$z_2$')
    
    return h

In [ ]:
from cycler import cycler

fig, ax = plt.subplots()

# Plot contour of target multivariate Gaussian:
X,Y = np.meshgrid(np.linspace(min(samples[:,0]), max(samples[:,0]), 1000), np.linspace(min(samples[:,1]), max(samples[:,1]),1000))
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

sm = plt.cm.ScalarMappable(cmap='viridis')
sm.set_clim(0, iters)
plt.colorbar(sm, ticks=np.linspace(0,iters,5), label='Iteration')

# ax.cla()
colors = [plt.cm.viridis(i) for i in np.linspace(0, 1, iters)]
ax.set_prop_cycle(cycler('color', colors))
h = []
for t in range(iters):
    h = plt.plot(samples[t:t+2,0], samples[t:t+2,1], marker='o')
ax.set_xlim(min(samples[:,0]), max(samples[:,0]))
ax.set_ylim(min(samples[:,1]), max(samples[:,1]))
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
# plt.legend(fontsize=fontsize)

# Generate animation from history of samples:
ani = animation.FuncAnimation(fig, animate, frames=iters, fargs=[{'ax':ax, 'samples':samples, 'pos':pos, 
                                                                  'target_mu':target_mu, 'target_sigma':target_sigma, 
                                                                  'X':X, 'Y':Y}], blit=True)
HTML(ani.to_jshtml())
print('Finished animation. Saving...')
ani.save('./Gibbs.gif', writer='imagemagick', fps=5, dpi=50)